In [1]:
import torch
!pip install transformers -q

from transformers import pipeline
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

from google.colab import files
import io

In [2]:
#get index for the maximum score from prediction in m
############################################################
def get_max_index(ld):
    lv = [d['score'] for d in ld]
    lvind = lv.index(max(lv))
    return lvind
############################################################

In [3]:
#get label with the maximum score
############################################################
def get_max_label(ld):
    val = ld['scores']
    val_ind = lv.index(max(lv))
    label = ld['labels'][val_ind]
    return label
############################################################

In [4]:
#local file upload
uploaded = files.upload()
rmp = pd.read_csv(io.BytesIO(uploaded['rmp_small.csv']))

Saving rmp_small.csv to rmp_small.csv


In [5]:
#checking the file
rmp

,professor_name,school_name,department_name,local_name,state_name,year_since_first_review,star_rating,take_again,diff_index,tag_professor,num_student,post_date,name_onlines,name_not_onlines,student_star,student_difficult,attence,for_credits,would_take_agains,grades,help_useful,help_not_useful,comments,word_comment,gender,race,asian,hispanic,nh_black,nh_white,gives_good_feedback,caring,respected,participation_matters,clear_grading_criteria,skip_class,amazing_lectures,inspirational,tough_grader,hilarious,get_ready_to_read,lots_of_homework,accessible_outside_class,lecture_heavy,extra_credit,graded_by_few_things,group_projects,test_heavy,so_many_papers,beware_of_pop_quizzes,IsCourseOnline
0,Leslie Looney,University Of Illinois at Urbana-Champaign,Astronomy department,Champaign\xe2\x80\x93Urbana,IL,11.0,4.7,NaN,2.0,Hilarious (2) GROUP PROJECTS (2) Gives good ...,26.0,06/27/2017,NAN,ASTR122,5.0,3.0,Mandatory,Yes,Yes,B,0.0,0.0,"This class is hard, but its a two-in-one gen-e...",44.0,unknown,hispanic,0.008548,0.731496,0.087095,0.172861,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1,Leslie Looney,University Of Illinois at Urbana-Champaign,Astronomy department,Champaign\xe2\x80\x93Urbana,IL,11.0,4.7,NaN,2.0,Hilarious (2) GROUP PROJECTS (2) Gives good ...,26.0,04/16/2017,NAN,ASTR122,5.0,2.0,Not Mandatory,NaN,Yes,A+,0.0,0.0,Definitely going to choose Prof. Looney\'s cla...,38.0,unknown,hispanic,0.008548,0.731496,0.087095,0.172861,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
2,Leslie Looney,University Of Illinois at Urbana-Champaign,Astronomy department,Champaign\xe2\x80\x93Urbana,IL,11.0,4.7,NaN,2.0,Hilarious (2) GROUP PROJECTS (2) Gives good ...,26.0,12/07/2016,NAN,ASTR330,4.0,3.0,NaN,Yes,Yes,NaN,0.0,0.0,I overall enjoyed this class because the assig...,32.0,unknown,hispanic,0.008548,0.731496,0.087095,0.172861,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
3,Leslie Looney,University Of Illinois at Urbana-Champaign,Astronomy department,Champaign\xe2\x80\x93Urbana,IL,11.0,4.7,NaN,2.0,Hilarious (2) GROUP PROJECTS (2) Gives good ...,26.0,12/08/2014,NAN,ASTR1008WKS,5.0,3.0,Mandatory,Yes,NaN,A,0.0,0.0,"Yes, it\'s possible to get an A but you\'ll de...",64.0,unknown,hispanic,0.008548,0.731496,0.087095,0.172861,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,Leslie Looney,University Of Illinois at Urbana-Champaign,Astronomy department,Champaign\xe2\x80\x93Urbana,IL,11.0,4.7,NaN,2.0,Hilarious (2) GROUP PROJECTS (2) Gives good ...,26.0,05/02/2014,NAN,ASTR150,5.0,1.0,Mandatory,NaN,NaN,NaN,0.0,0.0,Professor Looney has great knowledge in Astron...,57.0,unknown,hispanic,0.008548,0.731496,0.087095,0.172861,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,Dennis Richardson,Quinnipiac University,Biology department,Hamden,CT,15.0,4.0,NaN,3.5,BEWARE OF POP QUIZZES (4) Amazing lectures (3...,18.0,12/10/2013,NAN,BIO328,1.5,5.0,Not Mandatory,NaN,NaN,A-,1.0,1.0,Great sense of humor!!!! Love parasites now!!!!!,7.0,unknown,nh_white,0.088680,0.073583,0.013858,0.823878,0,1,1,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0
19996,Dennis Richardson,Quinnipiac University,Biology department,Hamden,CT,15.0,4.0,NaN,3.5,BEWARE OF POP QUIZZES (4) Amazing lectures (3...,18.0,04/17/2009,NAN,BIO102,2.5,4.0,NaN,NaN,NaN,NaN,0.0,1.0,he is a really nice guy and is really funny..h...,55.0,unknown,nh_white,0.088680,0.073583,0.013858,0.823878,0,1,1,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0
19997,Dennis Richardson,Quinnipiac University,Biology department,Hamden,CT,15.0,4.0,NaN,3.5,BEWARE OF POP QUIZZES (4) Amazing lectures (3...,18.0,10/28/2008,NAN,BIO326,5.0,3.0,NaN,NaN,NaN,NaN,1.0,0.0,His parasitology class is a lot of work but he...,22.0,unknown,nh_white,0.088680,0.073583,0.013858,0.823878,0,1,1,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0
19998,Dennis Richardson,Quinnipiac University,Biology department,Hamden,CT,15.0,

In [6]:
#main variables

sentences = list(rmp['comments'])
labels = list(rmp['student_star'])

In [7]:
#remove empty(NaN) from both lists
selnan = list()
for idx,name in enumerate(sentences):
    if name == '' or pd.isnull(name):
        selnan.append(idx)
sentences = [sentences[i] for i, e in enumerate(sentences) if i not in selnan]
labels = [labels[i] for i, e in enumerate(labels) if i not in selnan]

In [8]:
#shorterning senctences to reduce calculation time, remove [0:500] for the full amount

sentences_short = sentences[0:500]

In [9]:
#creating labels to calculate which qualities are mentioned

l1 = "simple tests"
l2 = "easy going"
l3 = "good social skills"
l4 = "good teaching skills"
l5 = "small amount of homework"
l6 = "course or professor review or opinion"
candidate_labels=[l1, l2, l3, l4, l5, l6]

In [10]:
classifier = pipeline("zero-shot-classification",
                      model = "facebook/bart-large-mnli",
                      device = 0

)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [11]:
prediction = classifier(sentences_short, candidate_labels=candidate_labels)

In [12]:
#sample

prediction[432]

{'sequence': "He is HORRIBLE!!!!! Do not take him if you can. His tests are very hard and nothing is multiple choice and he has other students grade his work because he can\\'t! He has even lost students work!",
 'labels': ['course or professor review or opinion',
  'simple tests',
  'good social skills',
  'small amount of homework',
  'easy going',
  'good teaching skills'],
 'scores': [0.6967049837112427,
  0.10814930498600006,
  0.06387168169021606,
  0.06054888293147087,
  0.03846772015094757,
  0.032257407903671265]}

In [13]:
lv = prediction[450]['scores']
lv

[0.7546281814575195,
 0.18923431634902954,
 0.03956247493624687,
 0.006358221173286438,
 0.005219459533691406,
 0.004997385200113058]

In [14]:
lvind = lv.index(max(lv))

In [15]:
l_max_labels = [get_max_label(val) for val in prediction]

In [16]:
df = pd.DataFrame(list(zip(l_max_labels, labels)), columns =['prof_opinion', 'score'])
df

,prof_opinion,score
0,course or professor review or opinion,5.0
1,easy going,5.0
2,easy going,4.0
3,course or professor review or opinion,5.0
4,easy going,5.0
...,...,...
495,good teaching skills,5.0
496,course or professor review or opinion,5.0
497,good teaching skills,5.0
498,course or professor review or opinion,5.0


In [17]:
#calculate crosstabulations
result = pd.crosstab(df["prof_opinion"], df["score"])
result

score,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
prof_opinion,,,,,,,,,
course or professor review or opinion,82,22,30,18,23,1,16,21,33
easy going,4,2,2,3,4,3,7,11,19
good social skills,4,1,2,2,2,1,5,7,7
good teaching skills,0,1,0,2,0,10,25,23,89
simple tests,1,1,0,2,0,0,3,0,2
small amount of homework,1,1,1,0,2,0,0,0,4


Testing different model to check if outcomes match

In [18]:

classifier = pipeline("zero-shot-classification",
                      model = "valhalla/distilbart-mnli-12-1",
                      device = 0
)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/890M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/890M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:0


In [19]:
prediction = classifier(sentences_short, candidate_labels=candidate_labels)

In [20]:
prediction[432]

{'sequence': "He is HORRIBLE!!!!! Do not take him if you can. His tests are very hard and nothing is multiple choice and he has other students grade his work because he can\\'t! He has even lost students work!",
 'labels': ['course or professor review or opinion',
  'simple tests',
  'good teaching skills',
  'easy going',
  'good social skills',
  'small amount of homework'],
 'scores': [0.5566353797912598,
  0.13322927057743073,
  0.09503643959760666,
  0.07391464710235596,
  0.07106410712003708,
  0.0701201781630516]}

In [21]:
lv = prediction[450]['scores']
lv

[0.7150219082832336,
 0.17812635004520416,
 0.04155725613236427,
 0.029008500277996063,
 0.021642424166202545,
 0.01464350800961256]

In [22]:
l_max_labels = [get_max_label(val) for val in prediction]

In [23]:
df = pd.DataFrame(list(zip(l_max_labels, labels)), columns =['prof_opinion', 'score'])
df

,prof_opinion,score
0,course or professor review or opinion,5.0
1,easy going,5.0
2,easy going,4.0
3,course or professor review or opinion,5.0
4,easy going,5.0
...,...,...
495,good teaching skills,5.0
496,good teaching skills,5.0
497,good teaching skills,5.0
498,good teaching skills,5.0


In [24]:
#calculate crosstabulations
result = pd.crosstab(df["prof_opinion"], df["score"])
result

score,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
prof_opinion,,,,,,,,,
course or professor review or opinion,62,19,27,16,19,1,16,21,46
easy going,15,6,4,7,5,4,10,13,26
good social skills,0,0,0,1,0,0,3,2,5
good teaching skills,12,2,4,3,4,10,23,25,72
simple tests,2,1,0,0,0,0,2,0,1
small amount of homework,1,0,0,0,3,0,2,1,4
